# Premier League Data Loading Pipeline
Load and process data from multiple sources:
- FBref (via soccerdata library) - Schedule, team stats, player stats
- Understat (CSV files) - Match-level xG data, player rosters

In [ ]:
%pip install soccerdata -q

In [ ]:
import os
import pandas as pd
import numpy as np
import soccerdata as sd
from datetime import datetime

# Configuration
LEAGUE = "ENG-Premier League"
SEASON = "2425"  # 2024/25 season
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

# Understat CSV paths
UNDERSTAT_TEAM_CSV = "./understat_matches_team.csv"
UNDERSTAT_ROSTER_CSV = "./understat_matches_roster.csv"

print(f"📅 Target Season: 2024/25")
print(f"🏆 League: {LEAGUE}")
print(f"📁 Data Directory: {DATA_DIR}")

## 1. Load FBref Data
Load season schedule, team stats, and player stats from FBref using soccerdata.

In [ ]:
print("="*60)
print("LOADING FBREF DATA")
print("="*60)

# Initialize FBref scraper with retry logic
try:
    fbref = sd.FBref(leagues=LEAGUE, seasons=SEASON)
    
    # Load schedule/fixtures
    print("\n📅 Loading match schedule...")
    schedule = fbref.read_schedule()
    
    # Filter to 2024/25 season only
    schedule = schedule[schedule.index.get_level_values('season') == SEASON].copy()
    
    # Reset index and save
    schedule_df = schedule.reset_index()
    schedule_df.to_csv(os.path.join(DATA_DIR, "fbref_schedule.csv"), index=False)
    print(f"✓ Schedule loaded: {len(schedule_df)} matches")
    
    # Load team season stats
    print("\n📊 Loading team stats...")
    team_stats = fbref.read_team_season_stats(stat_type="standard")
    team_stats = team_stats[team_stats.index.get_level_values('season') == SEASON].copy()
    team_stats_df = team_stats.reset_index()
    team_stats_df.to_csv(os.path.join(DATA_DIR, "fbref_team_stats.csv"), index=False)
    print(f"✓ Team stats loaded: {len(team_stats_df)} teams")
    
    # Load player season stats
    print("\n👤 Loading player stats...")
    player_stats = fbref.read_player_season_stats(stat_type="standard")
    player_stats = player_stats[player_stats.index.get_level_values('season') == SEASON].copy()
    player_stats_df = player_stats.reset_index()
    player_stats_df.to_csv(os.path.join(DATA_DIR, "fbref_player_stats.csv"), index=False)
    print(f"✓ Player stats loaded: {len(player_stats_df)} players")
    
    print("\n✅ FBref data loaded successfully!")
    fbref_available = True
    
except Exception as e:
    print(f"\n⚠️ FBref loading failed: {e}")
    print("Will proceed with Understat data only.")
    fbref_available = False
    schedule_df = pd.DataFrame()
    team_stats_df = pd.DataFrame()
    player_stats_df = pd.DataFrame()

## 2. Load Understat Match Data
Load match-level team and player data from Understat CSV files.

In [ ]:
print("\n" + "="*60)
print("LOADING UNDERSTAT DATA")
print("="*60)

# Load Understat match data
print("\n📊 Loading Understat match data...")
understat_matches = pd.read_csv(UNDERSTAT_TEAM_CSV)
understat_matches['date'] = pd.to_datetime(understat_matches['date'])

# Filter to 2024 season (2024/25 in Understat is year 2024)
understat_matches_2024 = understat_matches[understat_matches['season'] == 2024].copy()

print(f"✓ Understat matches loaded: {len(understat_matches_2024)} matches")
print(f"  Date range: {understat_matches_2024['date'].min()} to {understat_matches_2024['date'].max()}")

# Load player roster data
print("\n👤 Loading Understat player data...")
understat_players = pd.read_csv(UNDERSTAT_ROSTER_CSV)

# Extract match IDs and dates from team matches
match_dates = understat_matches[['id', 'date', 'season']].copy()
match_dates.columns = ['match_id', 'date', 'season']
match_dates['date'] = pd.to_datetime(match_dates['date'])

# Join with player data
understat_players['match_id'] = understat_players['match_link'].str.extract(r'/match/(\d+)').astype(int)
understat_players = understat_players.merge(match_dates, on='match_id', how='left')

# Filter to 2024 season
understat_players_2024 = understat_players[understat_players['season'] == 2024].copy()

print(f"✓ Understat player data loaded: {len(understat_players_2024)} player-match records")

print("\n✅ Understat data loaded successfully!")

## 3. Transform Match Data to Team Perspective
Convert wide-format match data to team-match format (one row per team per match).

In [ ]:
print("\n" + "="*60)
print("TRANSFORMING MATCH DATA")
print("="*60)

def transform_match_to_team_perspective(df):
    """Transform match data from wide to long format (team perspective)."""
    # Home team perspective
    home_df = pd.DataFrame({
        'match_id': df['id'],
        'date': pd.to_datetime(df['date']),
        'season': df['season'],
        'league': df['league'],
        'team_name': df['team_h'],
        'opponent': df['team_a'],
        'venue': 'Home',
        'goals_for': df['h_goals'],
        'goals_against': df['a_goals'],
        'xG': df['h_xg'],
        'xGA': df['a_xg'],
        'shots': df['h_shot'],
        'shots_against': df['a_shot'],
        'shots_on_target': df['h_shotOnTarget'],
        'shots_on_target_against': df['a_shotOnTarget'],
        'deep': df['h_deep'],
        'deep_allowed': df['a_deep'],
        'ppda': df['h_ppda'],
        'ppda_allowed': df['a_ppda'],
    })
    
    # Away team perspective
    away_df = pd.DataFrame({
        'match_id': df['id'],
        'date': pd.to_datetime(df['date']),
        'season': df['season'],
        'league': df['league'],
        'team_name': df['team_a'],
        'opponent': df['team_h'],
        'venue': 'Away',
        'goals_for': df['a_goals'],
        'goals_against': df['h_goals'],
        'xG': df['a_xg'],
        'xGA': df['h_xg'],
        'shots': df['a_shot'],
        'shots_against': df['h_shot'],
        'shots_on_target': df['a_shotOnTarget'],
        'shots_on_target_against': df['h_shotOnTarget'],
        'deep': df['a_deep'],
        'deep_allowed': df['h_deep'],
        'ppda': df['a_ppda'],
        'ppda_allowed': df['h_ppda'],
    })
    
    # Combine
    team_matches = pd.concat([home_df, away_df], ignore_index=True)
    team_matches = team_matches.sort_values(['date', 'match_id']).reset_index(drop=True)
    
    # Add derived columns
    team_matches['goal_diff'] = team_matches['goals_for'] - team_matches['goals_against']
    team_matches['result'] = team_matches.apply(
        lambda row: 'W' if row['goals_for'] > row['goals_against']
        else ('D' if row['goals_for'] == row['goals_against'] else 'L'),
        axis=1
    )
    team_matches['points'] = team_matches['result'].map({'W': 3, 'D': 1, 'L': 0})
    
    return team_matches

# Transform 2024/25 season data
print("\n🔄 Transforming to team-match format...")
team_matches_2024 = transform_match_to_team_perspective(understat_matches_2024)

print(f"✓ Team matches created: {len(team_matches_2024)} records")
print(f"  Unique teams: {team_matches_2024['team_name'].nunique()}")
print(f"  Matches per team: {len(team_matches_2024) / team_matches_2024['team_name'].nunique():.1f}")

# Save
team_matches_2024.to_csv(os.path.join(DATA_DIR, "team_matches_2024.csv"), index=False)
print(f"\n💾 Saved: team_matches_2024.csv")

## 4. Process Player Data
Transform player roster data with match information.

In [ ]:
print("\n" + "="*60)
print("PROCESSING PLAYER DATA")
print("="*60)

# Get team names from match data
match_teams = understat_matches_2024[['id', 'team_h', 'team_a']].copy()

# Process player data
players_2024 = understat_players_2024.copy()

# Add team names based on h_a indicator
players_2024 = players_2024.merge(
    match_teams,
    left_on='match_id',
    right_on='id',
    how='left',
    suffixes=('', '_match')
)

players_2024['team_name'] = players_2024.apply(
    lambda row: row['team_h'] if row['h_a'] == 'h' else row['team_a'],
    axis=1
)

players_2024['opponent'] = players_2024.apply(
    lambda row: row['team_a'] if row['h_a'] == 'h' else row['team_h'],
    axis=1
)

# Rename columns
players_2024 = players_2024.rename(columns={
    'time': 'minutes',
    'player': 'player_name',
})

# Select relevant columns
player_cols = [
    'match_id', 'date', 'season', 'player_id', 'player_name',
    'team_name', 'opponent', 'h_a', 'position', 'minutes',
    'goals', 'assists', 'xG', 'xA', 'shots', 'key_passes',
    'xGChain', 'xGBuildup', 'yellow_card', 'red_card'
]

players_2024 = players_2024[player_cols].sort_values(['date', 'match_id'])

print(f"✓ Player data processed: {len(players_2024)} player-match records")
print(f"  Unique players: {players_2024['player_name'].nunique()}")

# Save
players_2024.to_csv(os.path.join(DATA_DIR, "player_matches_2024.csv"), index=False)
print(f"\n💾 Saved: player_matches_2024.csv")

## 5. Load Historical Player Data (for context)
Load player stats from previous seasons for comparison and profiling.

In [ ]:
print("\n" + "="*60)
print("LOADING HISTORICAL PLAYER DATA")
print("="*60)

# Load all historical Understat player data (for player profiling context)
print("\n📚 Loading historical player data...")

# Get all seasons before 2024
historical_players = understat_players[understat_players['season'] < 2024].copy()

print(f"✓ Historical player data loaded: {len(historical_players)} records")
print(f"  Seasons: {sorted(historical_players['season'].unique())}")
print(f"  Unique players: {historical_players['player_name'].nunique()}")

# Save for reference
historical_players.to_csv(os.path.join(DATA_DIR, "player_matches_historical.csv"), index=False)
print(f"\n💾 Saved: player_matches_historical.csv")

## 6. Data Summary
Display summary statistics of loaded data.

In [ ]:
print("\n" + "="*60)
print("DATA LOADING SUMMARY")
print("="*60)

print("\n📊 2024/25 SEASON DATA:")
print(f"  Team Matches: {len(team_matches_2024):,}")
print(f"  Player Matches: {len(players_2024):,}")
print(f"  Unique Teams: {team_matches_2024['team_name'].nunique()}")
print(f"  Unique Players: {players_2024['player_name'].nunique()}")
print(f"  Date Range: {team_matches_2024['date'].min().date()} to {team_matches_2024['date'].max().date()}")

if fbref_available:
    print("\n📊 FBREF DATA:")
    print(f"  Matches: {len(schedule_df):,}")
    print(f"  Team Stats: {len(team_stats_df):,}")
    print(f"  Player Stats: {len(player_stats_df):,}")

print("\n📚 HISTORICAL DATA (for context):")
print(f"  Player Matches: {len(historical_players):,}")
print(f"  Seasons: {len(historical_players['season'].unique())}")

print("\n" + "="*60)
print("✅ DATA LOADING COMPLETE!")
print("="*60)

print("\n📁 Files saved in ./data/:")
print("  - team_matches_2024.csv")
print("  - player_matches_2024.csv")
print("  - player_matches_historical.csv")
if fbref_available:
    print("  - fbref_schedule.csv")
    print("  - fbref_team_stats.csv")
    print("  - fbref_player_stats.csv")

## Next Steps
Run the feature engineering notebooks to process this data further.